In [27]:
import pandas as pd
import os
import glob

# Base directory for the census data (adjusted for notebook path)
base_dir = '../data/raw/inegi_census'

# Find all CSV files across the 32 states
csv_pattern = os.path.join(base_dir, 'ageb_mza_urbana_*_cpv2020', 'conjunto_de_datos', 'conjunto_de_datos_ageb_urbana_*_cpv2020.csv')
csv_files = glob.glob(csv_pattern)

print(f"Found {len(csv_files)} CSV files (should be 32).")

# Initialize empty list for filtered DataFrames and a list for failed files
ageb_dfs = []
failed_files = []

# Loop through each CSV
for csv_path in csv_files:
    entity_code = os.path.basename(csv_path).split('_')[-2]  # Extract ENT code, e.g., '01' (fixed from -3 to -2)
    print(f"\nProcessing: {os.path.basename(csv_path)} (Entity {entity_code})")
    
    try:
        # Load CSV (handle UTF-8 BOM; try latin-1 if UTF-8 fails for accented chars)
        try:
            df = pd.read_csv(csv_path, encoding='utf-8-sig', low_memory=False)
        except UnicodeDecodeError:
            print(f"  UTF-8 failed—trying latin-1 for accents.")
            df = pd.read_csv(csv_path, encoding='latin-1', low_memory=False)
        
        # Debug: Check unique NOM_LOC values (first 10 to avoid spam)
        unique_nom_loc = df['NOM_LOC'].unique()[:10]
        print(f"  Unique NOM_LOC sample: {unique_nom_loc}")
        
        # Check if 'Total AGEB urbana' exists
        if 'Total AGEB urbana' in df['NOM_LOC'].values:
            # Filter for rows where NOM_LOC == 'Total AGEB urbana'
            ageb_totals = df[df['NOM_LOC'] == 'Total AGEB urbana'].copy()
            ageb_dfs.append(ageb_totals)
            print(f"  Found and added {len(ageb_totals)} matching rows.")
        else:
            print(f"  'Total AGEB urbana' NOT found in NOM_LOC—skipping this file.")
            failed_files.append((entity_code, csv_path, unique_nom_loc))
            
    except Exception as e:
        print(f"  Error loading file: {e}")
        failed_files.append((entity_code, csv_path, "Load error"))

# Report on failures
print(f"\n=== LOOP SUMMARY ===")
print(f"Successfully processed files with matches: {len(ageb_dfs)}")
if failed_files:
    print(f"\nFiles that broke the filter ({len(failed_files)}):")
    for code, path, sample in failed_files[:5]:  # Show first 5
        print(f"  - Entity {code}: {os.path.basename(path)} | Sample NOM_LOC: {sample}")
    if len(failed_files) > 5:
        print(f"  ... and {len(failed_files) - 5} more.")

# Attempt concat if any matches
if ageb_dfs:
    national_ageb = pd.concat(ageb_dfs, ignore_index=True)
    print(f"\nNational filtered shape: {national_ageb.shape}")
    print("First 5 rows:")
    print(national_ageb[['ENTIDAD', 'NOM_MUN', 'NOM_LOC', 'AGEB', 'MZA', 'POBTOT']].head())
    
    # Process AGEB ID: Standardize for merging (pad to 4 chars, ensure string)
    national_ageb['AGEB'] = national_ageb['AGEB'].astype(str).str.zfill(4)
    
    # Optional: Construct full CVEGEO (ENT + MUN + LOC + AGEB, padded) for advanced joins
    national_ageb['ENT'] = national_ageb['ENTIDAD'].astype(str).str.zfill(2)
    national_ageb['MUN'] = national_ageb['MUN'].astype(str).str.zfill(3)
    national_ageb['LOC'] = national_ageb['LOC'].astype(str).str.zfill(4)  # If LOC varies; drop if not used
    national_ageb['CVEGEO'] = (national_ageb['ENT'] + national_ageb['MUN'] + national_ageb['LOC'] + national_ageb['AGEB'])
    
    # Quick check: Unique AGEBs, sample
    print(f"\nUnique AGEBs after padding: {national_ageb['AGEB'].nunique()}")
    print("Sample with processed IDs:")
    print(national_ageb[['ENTIDAD', 'MUN', 'AGEB', 'CVEGEO', 'POBTOT']].head())
    
    # Save the national DF for next steps (optional, but recommended for efficiency)
    national_ageb.to_parquet('../data/processed/ageb_total/national_ageb.parquet', index=False)
    print("\nSaved as 'national_ageb.parquet' in processed/.")
else:
    print("\nNo matching files found—check if 'Total AGEB urbana' is the exact string (case-sensitive?).")
    print("Suggestion: Run df['NOM_LOC'].value_counts() on one file to inspect all unique values.")

Found 32 CSV files (should be 32).

Processing: conjunto_de_datos_ageb_urbana_10_cpv2020.csv (Entity 10)
  Unique NOM_LOC sample: ['Total de la entidad' 'Total del municipio'
 'Total de la localidad urbana' 'Total AGEB urbana' 'Canatlán' 'Canelas'
 'Coneto de Comonfort' 'Cuencamé de Ceniceros' 'Cuauhtémoc' 'Velardeña']
  Found and added 1890 matching rows.

Processing: conjunto_de_datos_ageb_urbana_19_cpv2020.csv (Entity 19)
  Unique NOM_LOC sample: ['Total de la entidad' 'Total del municipio'
 'Total de la localidad urbana' 'Total AGEB urbana' 'Abasolo' 'Agualeguas'
 'Los Aldamas' 'Ciudad de Allende' 'Anáhuac' 'Ciudad Apodaca']
  Found and added 2765 matching rows.

Processing: conjunto_de_datos_ageb_urbana_09_cpv2020.csv (Entity 09)
  Unique NOM_LOC sample: ['Total de la entidad' 'Total del municipio'
 'Total de la localidad urbana' 'Total AGEB urbana' 'Azcapotzalco'
 'Coyoacán' 'Cuajimalpa de Morelos' 'San Lorenzo Acopilco'
 'Gustavo A. Madero' 'Iztacalco']
  Found and added 2433 ma